# Import library

In [1]:
import sys
sys.path.append('src')
import enex_analysis as enex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dartwork_mpl as dm
import warnings
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
warnings.filterwarnings("ignore")

Load colors...
Load colormaps...


# Data

In [ ]:
T0_range = np.linspace(-20, 10, 100)
T_w_serv_range = np.linspace(30, 50, 100)

mat = np.zeros((len(T0_range), len(T_w_serv_range)))

Data = {'external unit': {}, 'refrigerant': {}, 'heat exchanger': {}, 'mixing valve': {}, 'whole': {}}

# External unit
Data['external unit']['X_eff'] = mat.copy()
Data['external unit']['E_fan'] = mat.copy()
Data['external unit']['X_a_ext_out'] = mat.copy()
Data['external unit']['X_a_ext_in'] = mat.copy()
Data['external unit']['dV_a_ext'] = mat.copy()


# Refrigerant
Data['refrigerant']['X_eff'] = mat.copy()
Data['refrigerant']['E_cmp'] = mat.copy()
Data['refrigerant']['X_r_ext'] = mat.copy()
Data['refrigerant']['X_r_exch'] = mat.copy()

# Heat exchanger
Data['heat exchanger']['X_eff'] = mat.copy()

# Mixing valve
Data['mixing valve']['X_eff'] = mat.copy()
Data['mixing valve']['X_w_serv'] = mat.copy()
Data['mixing valve']['X_w_exch'] = mat.copy()

# Whole
Data['whole']['Ex_eff'] = mat.copy()
Data['whole']['En_eff'] = mat.copy()
Data['whole']['COP'] = mat.copy()
Data['whole']['PLR'] = mat.copy()

# np.zeros((len(T0_range), len(T_w_serv_range)))
for i, T0 in enumerate(T0_range): # row
    for j, T_w_serv in enumerate(T_w_serv_range): # col
        ASHPB = enex.HeatPumpBoiler_without_tank()
        ASHPB.dV_w_serv = 10
        ASHPB.Q_r_max = 40000
        
        ASHPB.T0          = T0
        ASHPB.T_w_serv    = T_w_serv
        ASHPB.T_w_sup     = 0.2 * T0 + 10
        ASHPB.T_a_ext_out = T0 - 5
        ASHPB.T_r_ext     = T0 - 10
        
        ASHPB.T_r_exch    = T_w_serv + 10
        ASHPB.T_w_exch    = T_w_serv + 5
        ASHPB.system_update()
        
        # External unit exergy efficiency: X_a_ext_out / (E_fan + X_r_ext + X_a_ext_in)
        Data['external unit']['X_eff'][i, j] = (ASHPB.X_a_ext_out / (ASHPB.E_fan + ASHPB.X_r_ext + ASHPB.X_a_ext_in)) * 100
        Data['external unit']['E_fan'][i, j] = ASHPB.E_fan
        Data['external unit']['X_a_ext_out'][i, j] = ASHPB.X_a_ext_out
        Data['external unit']['X_a_ext_in'][i, j] = ASHPB.X_a_ext_in
        Data['external unit']['dV_a_ext'][i, j] = ASHPB.dV_a_ext

        # Refrigerant loop exergy efficiency: (X_r_exch + X_r_ext) / E_cmp
        Data['refrigerant']['X_eff'][i, j] = (ASHPB.X_r_exch + ASHPB.X_r_ext) / ASHPB.E_cmp * 100
        Data['refrigerant']['E_cmp'][i, j] = ASHPB.E_cmp
        Data['refrigerant']['X_r_ext'][i, j] = ASHPB.X_r_ext
        Data['refrigerant']['X_r_exch'][i, j] = ASHPB.X_r_exch

        # Mixing valve exergy efficiency: X_w_serv / (X_w_exch + X_w_sup_mix)
        Data['heat exchanger']['X_eff'][i, j] = (ASHPB.X_w_exch / (ASHPB.X_r_exch + ASHPB.X_w_sup_exch)) * 100
        
        # Mixing valve
        Data['mixing valve']['X_eff'][i, j] = (ASHPB.X_w_serv / (ASHPB.X_w_exch + ASHPB.X_w_sup_mix)) * 100
        
        # Whole
        Data['whole']['Ex_eff'][i, j] = ASHPB.X_eff*100
        Data['whole']['En_eff'][i, j] = (ASHPB.Q_w_serv/(ASHPB.E_fan + ASHPB.E_cmp))*100
        Data['whole']['COP'][i, j] = ASHPB.COP
        Data['whole']['PLR'][i, j] = ASHPB.Q_r_exch / ASHPB.Q_r_max * 100
        
print(np.max(Data['refrigerant']['X_eff']))
print(np.max((T0_range)*0.2 + 10), np.min((T0_range)*0.2 + 10))

92.24879632200104
12.0 6.0


# Figure

## Figure setting

In [68]:
plt.rcParams['font.size'] = 12

fs_dict = {
    'label': dm.fs(0),
    'tick': dm.fs(-1),
    'ctick': dm.fs(-2),
    'legend': dm.fs(-1.5),
    'annotation': dm.fs(-1),
            }

pad = {
    'label': 6,
    'tick': 4,
}

layout ={
    'bbox': (0.1, 0.1, 0.8, 0.8),
    'margins': (0, 0, 0, 0),
}

LW = np.arange(0.25, 3.0, 0.25)

## Figure function

In [69]:
def generate_two_heatmap_with_each_cbar(datas,
                                        cmaps=None,
                                        cmaxs=None,
                                        cmins=None,
                                        cints=None,
                                        vmins=None,
                                        vmaxs=None,
                                        contour_colors=None,
                                        contour_levels=None,
                                        fmts=None,
                                        clabels=None,
                                        annotations=None,
                                        fig_name=None,
                                        save=None
):
    """
    Generates and displays a figure with two side-by-side heatmaps, each with its own colorbar.
    
    Most parameters have default values and can be omitted unless customization is needed.
    """
    # Set default values for optional parameters if they are not provided
    if cmaps is None: cmaps = ['viridis', 'plasma']
    if cmins is None: cmins = [np.min(d) for d in datas]
    if cmaxs is None: cmaxs = [np.max(d) for d in datas]
    if cints is None: cints = [(cmaxs[i] - cmins[i]) / 5 for i in range(2)]
    if vmins is None: vmins = cmins
    if vmaxs is None: vmaxs = cmaxs
    if contour_colors is None: contour_colors = ['dm.gray0', 'dm.gray0']
    if contour_levels is None: contour_levels = [np.linspace(cmins[i], cmaxs[i], 5) for i in range(2)]
    if fmts is None: fmts = ['%1.1f', '%1.1f']
    if clabels is None: clabels = ['Value', 'Value']
    if annotations is None: annotations = ['(a)', '(b)']
    if fig_name is None: fig_name = 'default_heatmap_figure'
    if save is None: save = False
    
    # Plot ====================================================================================================
    nrows = 1
    ncols = 2
    nfigs = nrows * ncols

    fig, ax = plt.subplots(
        nrows, ncols, 
        sharex=True, sharey=True, 
        figsize=(dm.SW * 2.5, dm.SW),  # 가로 두배
        facecolor='none', edgecolor='k',
        squeeze=False,
        dpi=300,
    )

    # x축을 T_w_serv, y축을 T0로 변경
    xmin = [np.min(T_w_serv_range)] * nfigs
    xmax = [T_w_serv_range[-1]] * nfigs
    xint = [5] * nfigs
    xmar = [0] * nfigs

    ymin = [np.min(T0_range)] * nfigs
    ymax = [T0_range[-1]] * nfigs
    yint = [10] * nfigs
    ymar = [0] * nfigs

    norms  = [mcolors.Normalize(vmin=vmins[0], vmax=vmaxs[0]), mcolors.Normalize(vmin=vmins[1], vmax=vmaxs[1])]

    ims = []
    css = []

    for i in range(nrows):
        for j in range(ncols):
            idx = i * ncols + j
            im = ax[i, j].imshow(
                datas[idx],
                cmap=cmaps[idx],
                norm=norms[idx],
                extent=[T_w_serv_range[0], T_w_serv_range[-1], T0_range[0], T0_range[-1]],
                aspect='auto',
                interpolation='none',
                origin='lower',
            )
            ims.append(im)
            
            ax[i, j].set_xlim(xmin[idx] - xmar[idx], xmax[idx] + xmar[idx])
            ax[i, j].set_ylim(ymin[idx] - ymar[idx], ymax[idx] + ymar[idx])
            
            ax[i, j].set_xlabel('Service hot water temperature [°C]', fontsize=fs_dict['label'])
            ax[i, 0].set_ylabel('Environmental temperature [°C]', fontsize=fs_dict['label'])
            
            ax[i, j].annotate(annotations[idx], xy=(.01, 1.01), xycoords='axes fraction',
                horizontalalignment='left', verticalalignment='bottom', fontsize=fs_dict['annotation']) 
            
            ax[i, j].tick_params(direction='in', labelsize=fs_dict['label'], which='major', length=2.5, width=0.5  , pad=pad['tick'])
            ax[i, j].tick_params(direction='in', labelsize=fs_dict['label'], which='minor', length=1.25, width=0.25, pad=pad['tick'])
            
            ax[i, j].set_xticks(np.arange(xmin[idx], xmax[idx]*1.001, xint[idx]))
            ax[i, j].set_yticks(np.arange(ymin[idx], ymax[idx]*1.001, yint[idx]))
            
            ax[i, j].xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
            ax[i, j].yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
            
            ax[i, j].grid(False)

            for k in ['top', 'bottom', 'left', 'right']:
                ax[0, j].spines[k].set_visible(True)
                ax[0, j].spines[k].set_linewidth(0.5)
                ax[0, j].spines[k].set_color('k')

            cs = ax[i, j].contour(
                T_w_serv_range, T0_range, datas[idx], 
                levels=contour_levels[idx], 
                colors=contour_colors[idx], linewidths=1, linestyles='solid'
            )
            
            css.append(cs)
            ax[i, j].clabel(cs, fmt=fmts[idx], colors=contour_colors[idx], fontsize=fs_dict['tick'])

    # Increase horizontal spacing between subplots
    fig.subplots_adjust(wspace=0.2)
    dm.simple_layout(fig, bbox=[0.01, 0.8, 0.01, 1.0], margins=[0.0, 0.1, 0.05, 0.05])

    # colorbar (Supplied cool exergy)
    cbar_width = 0.02  # 너비를 반으로 줄임
    cbar_dist_h = 0.04  # ax 우측에 가깝게

    bbox1 = ax[0, 1].get_position()
    cb_ax1 = fig.add_axes([bbox1.x1 + cbar_dist_h, bbox1.y0, cbar_width, bbox1.y1 - bbox1.y0])
    cbar1 = fig.colorbar(ims[0], cax=cb_ax1, ax=ax[0, 0], orientation='vertical')
    cbar1.ax.tick_params(which="major", direction='in', labelsize=fs_dict['ctick'], length=2, width=0.25, pad=pad['tick'])  # pad 값 증가
    cbar1.ax.minorticks_off()
    cbar1.set_ticks(np.arange(cmins[0], cmaxs[0]*1.001, cints[0]))
    cbar1.ax.set_ylabel(clabels[0], fontsize=fs_dict['label'], labelpad =pad['label'])
    cbar1.update_ticks()
    cbar1.outline.set_linewidth(0.5)

    # colorbar (Supplied cool exergy)
    cbar_width = 0.02  # 너비를 반으로 줄임
    cbar_dist_h += 0.10  # ax 우측에 가깝게
    
    # colorbar (COP)
    bbox2 = ax[0, 1].get_position()
    cb_ax2 = fig.add_axes([bbox2.x1 + cbar_dist_h, bbox2.y0, cbar_width, bbox2.y1 - bbox2.y0])
    cbar2 = fig.colorbar(ims[1], cax=cb_ax2, ax=ax[0, 1], orientation='vertical')
    cbar2.ax.tick_params(which="major", direction='in', labelsize=fs_dict['ctick'], length=2, width=0.25, pad=pad['tick'])  # pad 값 증가
    cbar2.ax.minorticks_off()
    cbar2.set_ticks(np.arange(cmins[1], cmaxs[1]*1.001, cints[1]))
    cbar2.ax.set_ylabel(clabels[1], fontsize=fs_dict['label'], labelpad =pad['label'])
    cbar2.update_ticks()
    cbar2.outline.set_linewidth(0.5)

    save_path = 'figure/'
    if save:
        plt.savefig(save_path + fig_name + '.png', dpi=600)
        plt.savefig(save_path + fig_name + '.svg', transparent=True)
    plt.close()
    dm.save_and_show(fig)

## Energy use of External unit & Refrigerant

In [70]:
W2kW = 1/1000
generate_two_heatmap_with_each_cbar(
    datas=[Data['external unit']['E_fan']*W2kW, Data['refrigerant']['E_cmp']*W2kW],
    cmaps=['dm.YlGn', 'dm.YlGn'],
    cmaxs=[1200*W2kW, 14000*W2kW],
    cints=[200*W2kW, 2000*W2kW],
    cmins=[0, 0],
    vmaxs=[1200*W2kW, 14000*W2kW],
    vmins=[0, 0],
    contour_levels=[
        np.array([i * 200*W2kW for i in range(10)]),
        np.array([i * 2000*W2kW for i in range(7)])
    ],
    contour_colors=['dm.gray0', 'dm.gray9'],
    fmts=['%1.1f', '%1.0f'],
    clabels=['Fan energy use [kW]', 'Compressor energy use [kW]'],
    annotations=['(a) External Unit', '(b) Refrigerant'],
    fig_name='ASHPB_energy_use_plots',
    save=True
)


In [71]:
np.max(Data['refrigerant']['E_cmp'])

np.float64(22646.599623215378)

## Exergy efficiency of External unit, Refrigerant, Heat exchanger

In [72]:
# Plot ====================================================================================================
nrows = 1
ncols = 3
nfigs = nrows * ncols

fig, ax = plt.subplots( 
    nrows, ncols, 
    sharex=True, sharey=True, 
    figsize=(dm.SW*3.5, dm.SW), 
    facecolor='none', edgecolor='k',
    squeeze=False,
    dpi=300,
)
# tick settings 
xmin = [np.min(T_w_serv_range)] * nfigs
xmax = [T_w_serv_range[-1]] * nfigs
xint = [5] * nfigs
xmar = [0] * nfigs

ymin = [np.min(T0_range)] * nfigs
ymax = [T0_range[-1]] * nfigs
yint = [10] * nfigs
ymar = [0] * nfigs
cmax = 90

im_list = []
cs_list = []

data_list = [Data['external unit']['X_eff'], Data['refrigerant']['X_eff'], Data['heat exchanger']['X_eff'],]

levels_list = [np.arange(8, 11, 0.2),
               np.arange(0, 100, 10),
               np.arange(0, 100, 10)]

color_list = ['dm.gray8', 'dm.gray1', 'dm.gray1']

eff_keys = ['external unit', 'refrigerant', 'heat exchanger']
subplot_titles = ['(a) External Unit', '(b) Refrigerant', '(c) heat exchanger']

for ridx in range(nrows):
    for cidx in range(ncols): 
        idx = cidx + ridx * ncols
        norm = mcolors.Normalize(vmin=0, vmax=cmax)
        eff_mat = data_list[idx]

        im = ax[ridx, cidx].imshow(
            eff_mat,
            cmap='dm.BuPu',
            norm=norm,
            extent=[T_w_serv_range[0], T_w_serv_range[-1], T0_range[0], T0_range[-1]],
            aspect='auto',
            interpolation='none',
            origin='lower',
        )
        im_list.append(im)
        
        ax[ridx, cidx].set_xlim(xmin[idx] - xmar[idx], xmax[idx] + xmar[idx])
        ax[ridx, cidx].set_ylim(ymin[idx] - ymar[idx], ymax[idx] + ymar[idx])

        ax[0, cidx].set_xlabel('Service hot water temperature [°C]', fontsize=fs_dict['label'])
        ax[ridx, 0].set_ylabel('Environmental temperature [°C]', fontsize=fs_dict['label'])

        ax[ridx, cidx].annotate(subplot_titles[idx], xy=(.01, 1.01), xycoords='axes fraction',
            horizontalalignment='left', verticalalignment='bottom', fontsize=fs_dict['annotation']) 

        ax[ridx, cidx].tick_params(direction='in', labelsize=fs_dict['label'], which='major', length=2.5, width=0.5  , pad = pad['tick'])
        ax[ridx, cidx].tick_params(direction='in', labelsize=fs_dict['label'], which='minor', length=1.25, width=0.25, pad = pad['tick'])

        ax[ridx, cidx].set_xticks(np.arange(xmin[idx], xmax[idx]*1.001, xint[idx]))
        ax[ridx, cidx].set_yticks(np.arange(ymin[idx], ymax[idx]*1.001, yint[idx]))

        ax[ridx, cidx].xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
        ax[ridx, cidx].yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
        ax[ridx, cidx].grid(False)

        for k in ['top', 'bottom', 'left', 'right']:
            ax[ridx, cidx].spines[k].set_visible(True)
            ax[ridx, cidx].spines[k].set_linewidth(0.5)
            ax[ridx, cidx].spines[k].set_color('k')

        cs = ax[ridx, cidx].contour(
            T_w_serv_range, T0_range, data_list[idx], 
            levels=levels_list[idx], 
            colors=color_list[idx], linewidths=1, linestyles='solid'
        )
        
        cs_list.append(cs)
        ax[ridx, cidx].clabel(cs, fmt=['%1.1f','%1.0f','%1.0f'][idx], colors=color_list[idx], fontsize=fs_dict['tick'])

dm.simple_layout(fig, bbox=[0.01, 0.95, 0.0, 1.0], margins=[0.0, 0.1, 0.05, 0.05])

# colorbar (공통 colorbar 하나만 추가)
bbox1 = ax[0, 0].get_position()
bbox2 = ax[0, 2].get_position()
# bbox2 = ax[1, 0].get_position()
# bbox4 = ax[1, 1].get_position()

# colorbar
cbar_width  = 0.016 #vertical  
cbar_height = 0.015 #horizontal  
cbar_dist_v = 0.12; # vertical colorbar distance from bbox edge
cbar_dist_h = 0.02; # horizontal colorbar distance from bbox edge


cb_ax1 = fig.add_axes([bbox2.x1 + cbar_dist_h, bbox2.y0, cbar_width, bbox2.y1 - bbox1.y0]) #[x_origin, y_origin, width, height]
cbar1 = fig.colorbar(im, cax=cb_ax1, ax=ax[0,0], orientation='vertical')
cbar1.ax.tick_params(which="major", direction='in', labelsize=fs_dict['ctick'], length=2, width=0.25, pad=pad['tick'])
cbar1.ax.minorticks_off()
cbar1.set_ticks(np.arange(0, cmax+1, 15))
cbar1.ax.set_ylabel('Exergy efficiency [%]', fontsize=fs_dict['label'], labelpad=pad['label'], )
cbar1.update_ticks()
cbar1.outline.set_linewidth(0.5)

plt.savefig('figure/ASHPB_exergy_efficiency_plots.png', dpi=600)
plt.savefig('figure/ASHPB_exergy_efficiency_plots.svg', transparent=True)
plt.close()
dm.save_and_show(fig)


## Refrigerant cool and warm exergy

In [51]:
W2kW = 1/1000
generate_two_heatmap_with_each_cbar(
    datas=[Data['refrigerant']['X_r_ext'] * W2kW, Data['refrigerant']['X_r_exch'] * W2kW],
    cmaps=['dm.BuPu', 'dm.BuPu'],
    cmaxs=[1000 * W2kW, 9000 * W2kW],
    cints=[200 * W2kW, 1500 * W2kW],
    cmins=[0, 0],
    vmaxs=[1000 * W2kW, 9000 * W2kW],
    vmins=[0, 0],
    contour_levels=[
        np.array([i * 100 * W2kW for i in range(10)]), np.array([i * 1500 * W2kW for i in range(7)])
    ],
    fmts=['%1.2f', '%1.1f'],
    clabels=['Supplied cool exergy [kW]', 'Supplied warm exergy [kW]'],
    annotations=['(a) Supplied cool exergy [kW]', '(b) Supplied warm exergy [kW]'],
    fig_name='ASHPB_COP&Supplied cool exergy_plots',
    save=True
)

In [52]:
np.max(Data['refrigerant']['X_r_ext']), np.max(Data['refrigerant']['X_r_exch']) 

(np.float64(659.057245048818), np.float64(6045.0771055753285))

## PLR & COP of Refrigerant

In [73]:
generate_two_heatmap_with_each_cbar(
    datas=[Data['whole']['PLR'], Data['whole']['COP']],
    cmaps=['dm.BuPu', 'dm.RdPu'],
    cmaxs=[100, 8],
    cints=[20, 1],
    cmins=[0, 0],
    vmaxs=[100, 8],
    vmins=[0, 0],
    contour_levels=[
        np.array([i * 10 for i in range(10)]),np.array([i for i in range(10)])
    ],
    fmts=['%1.0f', '%1.0f'],
    clabels=['PLR [%]', 'COP [-]'],
    annotations=['(a) PLR', '(b) COP'],
    fig_name='ASHPB_COP&PLR_plots',
    save=True
)


## Volumetric flow rate and Energy use of External unit

In [74]:
W2kW = 1/1000
generate_two_heatmap_with_each_cbar(
    datas=[Data['external unit']['dV_a_ext'], Data['external unit']['E_fan'] * W2kW],
    cmaps=['dm.Blues9', 'dm.YlGn'],
    cmaxs=[4, 1400 * W2kW],
    cints=[0.5, 200 * W2kW],
    cmins=[0, 0],
    vmaxs=[4, 1400 * W2kW],
    vmins=[0, 0],
    contour_levels=[np.arange(0, 4, 0.5), np.arange(0, 1400, 200) * W2kW],
    fmts=['%1.2f', '%1.1f'],
    clabels=['External unit discharge flow [m³/h]', 'Fan power use [kW]'],
    annotations=['(a) External unit discharge flow', '(b) Fan power use'],
    fig_name='ASHPB_external_unit_discharge_flow_and_fan_power',
    save=True
)

In [75]:
np.max(Data['external unit']['dV_a_ext']), np.max(Data['external unit']['E_fan'])

(np.float64(3.2903262559300424), np.float64(1096.7754186433476))

## Energy and Exergy efficiency of whole system

In [81]:
generate_two_heatmap_with_each_cbar(
    datas=[Data['whole']['En_eff'], Data['whole']['Ex_eff']],
    cmaps=['dm.YlGn', 'dm.BuPu'],
    cmaxs=[600, 40],
    cints=[100, 8],
    cmins=[0, 0],
    vmaxs=[600, 40],
    vmins=[0, 0],
    contour_levels=[np.arange(0, 600, 50), np.arange(0, 44, 4)],
    fmts=['%1.0f', '%1.0f'],
    clabels=['Energy efficiency [%]', 'Exergy efficiency [%]'],
    annotations=['(a) Energy efficiency', '(b) Exergy efficiency'],
    fig_name='Energy_and_Exergy_eff',
    save=True
)

# Water fall chart